#Задание
1. Использовать датасет MovieLens.

2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:
* TF-IDF на тегах и жанрах;

* средние оценки (+ median, variance и т. д.) пользователя и фильма.

3. Оценить RMSE на тестовой выборке.

#Датасет

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2024-01-30 07:27:05--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 108.177.125.101, 108.177.125.138, 108.177.125.113, ...
Connecting to drive.google.com (drive.google.com)|108.177.125.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-01-30 07:27:06--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.23.132, 2404:6800:4008:c02::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.23.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.01s   

2024-01-30 07:27:08 (73.2 MB/s)

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [ ]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
#Узнаем среднюю оценку
np.mean(ratings['rating'])

3.501556983616962

In [ ]:
#Узнаем медиану оценки
np.median(ratings['rating'])

3.5

In [ ]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


Соединяем жанры и теги в один датафрейм

In [ ]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')

In [ ]:
movies_with_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


# Объединим всё в один датафрейм

In [ ]:
movies_with_tags_and_ratings = pd.merge(movies_with_tags, ratings, on=['userId', 'movieId'], how='inner')

In [ ]:
movies_with_tags_and_ratings

,movieId,title,genres,userId,tag,timestamp_x,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,4.0,1122227329
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09,4.0,978575760
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09,3.5,1525286001
3,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09,4.0,1528843890
4,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09,4.0,1528843890
...,...,...,...,...,...,...,...,...
3471,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62.0,star wars,1.528935e+09,4.0,1528934550
3472,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,anime,1.537099e+09,3.5,1537098554
3473,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,comedy,1.537099e+09,3.5,1537098554
3474,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,gintama,1.537099e+09,3.5,1537098554


In [ ]:
# Объединяем теги и жанры в одну строку
movies_with_tags_and_ratings['tags_and_genres'] = movies_with_tags_and_ratings['tag'] + ' ' + movies_with_tags_and_ratings['genres']

In [ ]:
movies_with_tags_and_ratings

,movieId,title,genres,userId,tag,timestamp_x,rating,timestamp_y,tags_and_genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,4.0,1122227329,pixar Adventure|Animation|Children|Comedy|Fantasy
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09,4.0,978575760,pixar Adventure|Animation|Children|Comedy|Fantasy
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09,3.5,1525286001,fun Adventure|Animation|Children|Comedy|Fantasy
3,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09,4.0,1528843890,fantasy Adventure|Children|Fantasy
4,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09,4.0,1528843890,magic board game Adventure|Children|Fantasy
...,...,...,...,...,...,...,...,...,...
3471,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62.0,star wars,1.528935e+09,4.0,1528934550,star wars Action|Adventure|Children|Sci-Fi
3472,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,anime,1.537099e+09,3.5,1537098554,anime Action|Animation|Comedy|Sci-Fi
3473,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,comedy,1.537099e+09,3.5,1537098554,comedy Action|Animation|Comedy|Sci-Fi
3474,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,gintama,1.537099e+09,3.5,1537098554,gintama Action|Animation|Comedy|Sci-Fi


## Преобразование жанров и тегов в векторы

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Создаем TF-IDF матрицу на основе тегов и жанров
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_with_tags_and_ratings['tags_and_genres'])

In [ ]:
tfidf_matrix

<3476x1712 sparse matrix of type '<class 'numpy.float64'>'
	with 15103 stored elements in Compressed Sparse Row format>

In [ ]:
movies_with_tags_and_ratings_X = movies_with_tags_and_ratings.drop(columns = ['rating', 'timestamp_x', 'timestamp_y'], axis = 1)

In [ ]:
movies_with_tags_and_ratings_X

,movieId,title,genres,userId,tag,tags_and_genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,pixar Adventure|Animation|Children|Comedy|Fantasy
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,pixar Adventure|Animation|Children|Comedy|Fantasy
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,fun Adventure|Animation|Children|Comedy|Fantasy
3,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,fantasy Adventure|Children|Fantasy
4,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,magic board game Adventure|Children|Fantasy
...,...,...,...,...,...,...
3471,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62.0,star wars,star wars Action|Adventure|Children|Sci-Fi
3472,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,anime,anime Action|Animation|Comedy|Sci-Fi
3473,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,comedy,comedy Action|Animation|Comedy|Sci-Fi
3474,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,gintama,gintama Action|Animation|Comedy|Sci-Fi


In [ ]:
print(tfidf_matrix.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Обучение линейной регрессии

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
X = tfidf_matrix

In [ ]:
y = movies_with_tags_and_ratings.rating

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [ ]:
# Предсказание оценок на тестовом наборе
y_pred = model.predict(X_test)
y_pred

array([3.66425554, 3.22964924, 4.26823748, 3.71035257, 4.19215974,
       4.06082978, 3.81637169, 4.2759484 , 3.75805027, 3.50000607,
       4.08825384, 5.06513713, 3.4094905 , 4.5843022 , 3.7283339 ,
       3.97213826, 3.73590639, 4.51390734, 3.76458257, 4.54304708,
       3.95245913, 3.65450688, 4.285659  , 3.99994517, 2.55865573,
       4.30349679, 4.54225099, 3.97434947, 2.16208156, 3.51191894,
       3.41859477, 3.93341797, 1.24997998, 4.29702375, 4.03891504,
       3.7283339 , 4.5548885 , 3.59689745, 3.75901596, 4.30734184,
       4.055663  , 3.88286939, 4.54161681, 4.14597605, 4.11964091,
       3.55070199, 3.97890977, 3.95336875, 3.46145288, 3.69849093,
       4.02572645, 5.18684699, 3.52239532, 3.84758109, 3.68660548,
       3.77909437, 4.65048912, 4.04730504, 3.07522866, 2.9346384 ,
       4.21485548, 4.48593067, 2.64166951, 3.29236538, 4.6542502 ,
       3.80622996, 4.01444369, 3.72008903, 3.90967168, 3.690505  ,
       3.59689745, 4.56350761, 3.61785355, 4.21518283, 4.32876

## Оценка MSE

In [ ]:
# Оценка качества модели (среднеквадратичная ошибка)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.72861603472315
